In [1]:
import os

import numpy as np
import scipy as sc
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, MaxNLocator

torch.manual_seed(17)

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])

train_set = torchvision.datasets.FashionMNIST(root="../data/", train=True, download=True, transform=transform)
train_set_size = len(train_set)

test_set = torchvision.datasets.FashionMNIST(root="../data/", train=False, download=True, transform=transform)
test_set_size = len(test_set)

In [47]:
reduction_factor = 0.1
red_set, _ = torch.utils.data.random_split(train_set, (reduction_factor, 1.-reduction_factor))

In [55]:
train_set.__getitem__(red_set.indices)

TypeError: __getitem__() takes 2 positional arguments but 6001 were given